# Comparativa de Datos Históricos entre APIs

---

## **Contexto**:
Obtendremos **datos históricos** de dos servicios de API: **Open-Meteo** y **OpenWeatherMap**, para el **Puerto de La Cruz, España**. Los datos incluirán las siguientes variables: **temperatura**, **humedad**, **velocidad del viento**, y **precipitaciones**. Se analizarán las diferencias entre las mediciones proporcionadas por ambas fuentes para el mismo período (últimos 15 días) y se realizarán comparaciones gráficas.

---

## **Objetivos**:
1. **Consultar y procesar datos históricos** de Open-Meteo y OpenWeatherMap.
2. **Encapsular código en funciones** para su reutilización
3. **Crear series temporales** con las variables seleccionadas.
4. Realizar una **comparación gráfica** de las mediciones entre ambas fuentes.
5. **Analizar las diferencias** mediante estadísticas descriptivas y visualizaciones.

---

## **Requisitos Previos**:
- **API Key** de OpenWeatherMap.
- **Open-Meteo** no requiere autenticación.
- Instalar las librerías necesarias:


In [ ]:

! pip install requests pandas matplotlib seaborn



---

## Solución del ejercicio

### Consultar datos históricos desde las APIs

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Coordenadas de Puerto de La Cruz
LAT, LON = 28.4161, -16.5446
DAYS = 15  # Últimos 15 días
END_DATE = datetime.now().date()- timedelta(days = 3)
START_DATE = END_DATE - timedelta(days=DAYS)

# Open-Meteo API Configuración
OM_URL = "https://api.open-meteo.com/"

# AEMET API Configuración
AEMET_API_KEY = "TU_CLAVE_API" # Reemplaza con tu API Key
AEMET_URL = "https://opendata.aemet.es/opendata/"

### Encapsular código en funciones

In [ ]:
# Función para obtener datos de Open-Meteo
def obtener_datos_historicos_open_meteo(start_date, end_date, lat, lon, diarios=False):
    url =  "https://archive-api.open-meteo.com/v1/archive"
    print(url)
    response = requests.get(
        url=url,
        params={
            "latitude": lat,
            "longitude": lon,
            "start_date": start_date.strftime("%Y-%m-%d"),
            "end_date": end_date.strftime("%Y-%m-%d"),
            "hourly": "temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m"
        }
    )
    # print(response.request.url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error al obtener datos de Open-Meteo")
        return None

In [ ]:
datos_om = obtener_datos_historicos_open_meteo(START_DATE, END_DATE, LAT, LON)
datos_om

In [ ]:
# Listar estaciones AEMET
def listar_estaciones_aemet(key):
    headers = {"api_key": key}
    url = AEMET_URL+"api/valores/climatologicos/inventarioestaciones/todasestaciones"
    print(url)
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        d = response.json()
        if 'datos' in d:
            data = requests.get(d['datos'])
            if data.status_code == 200:
                return data.json()
            else:
                print("Error en la respuesta a consulta a AEMET:", data.status_code)        
        return 
    else:
        print("Error en la consulta a AEMET:", response.status_code)
        return None
    

In [ ]:
from urllib.parse import quote

# Obtener datos de AEMET APi
def obtener_datos_historicos_aemet(key, start_date, end_date, id):
    variables = ['']
    headers = {"api_key": key}
    # Formatear fechas
    fecha_ini = start_date.strftime('%Y-%m-%dT%H:%M:%SUTC')
    fecha_fin = end_date.strftime('%Y-%m-%dT%H:%M:%SUTC')
    url = AEMET_URL+"/api/valores/climatologicos/diarios/datos/fechaini/{}/fechafin/{}/estacion/{}".format(fecha_ini, fecha_fin, id)
    print(url)

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        if not 'datos' in response.json():
            print("Error al descarga datos AEMET: Consulta vacía")
            return None
        data_url = response.json()["datos"]
        data_response = requests.get(data_url)
        if data_response.status_code == 200:
            data = data_response.json()
            return data
        else:
            print("Error al descargar datos AEMET:", data_response.status_code)
            return None
    else:
        print("Error en la consulta a AEMET:", response.status_code)
        return None

In [ ]:
# Listar estaciones AEMET
estaciones_aemet = listar_estaciones_aemet(AEMET_API_KEY)
print(len(estaciones_aemet))
estaciones_aemet[0]

In [ ]:
# Filtrar estaciones de S/C de Tenerife
estaciones_tf = [item for item in estaciones_aemet if "tenerife" in item["provincia"].lower()]
print(len(estaciones_tf))
estaciones_tf[0]

In [ ]:
# Filtrar aquellas cuyo nombre tenga la el texto 'cruz'
estaciones_pc = [item for item in estaciones_tf if "cruz" in item["nombre"].lower()]
estaciones_pc

In [ ]:
# Obtener datos de aemet para la estación seleccionada
datos_aemet = obtener_datos_historicos_aemet(AEMET_API_KEY, START_DATE, END_DATE, 'C459Z')
datos_aemet[0]

In [ ]:
# Obtener los datos openmeteo
datos_om = obtener_datos_historicos_open_meteo(START_DATE, END_DATE, LAT, LON)
datos_om

---

#### **Paso 2: Procesar los datos y convertirlos en DataFrames**

In [ ]:
# Procesar datos de Open-Meteo
agregar_diarios=True
# Convertir el JSON a DataFrame
df = pd.DataFrame(datos_om['hourly'])
df['time'] = pd.to_datetime(df['time'])
df.rename(columns={"time": "Fecha"}, inplace=True)
df.set_index('Fecha', inplace=True)
if agregar_diarios:
    df['Dia'] = df.index.date
    daily_df = df.groupby('Dia').agg(
        temperature = ('temperature_2m', 'mean'),  # Promedio de temperatura
        temperature_min = ('temperature_2m', 'min'),  # Minimo de temperatura
        temperature_max = ('temperature_2m', 'max'),  # Máximo de temperatura
        humidity =('relative_humidity_2m', 'mean'),  # Promedio de humedad relativa
        humidity_min =('relative_humidity_2m', 'min'),  # Mínimo de humedad relativa
        humidity_max =('relative_humidity_2m', 'max'),  # Máximo de humedad relativa
        precipitation = ('precipitation', 'sum'),  # Suma total de precipitaciones
        wind_speed = ('wind_speed_10m', 'mean')  # Promedio de velocidad del viento
    ).reset_index()
    daily_df.rename(columns={"Dia": "Fecha"}, inplace=True)
    daily_df.set_index(pd.to_datetime(daily_df['Fecha']), inplace=True)
    daily_df = daily_df.drop("Fecha", axis=1)
    df_om = daily_df
else:
    df_om = df

# Procesar datos de AEMET
df = pd.DataFrame(datos_aemet)
df['Fecha'] = pd.to_datetime(df['fecha'])
df.set_index("Fecha", inplace=True)
df = df[['tmed', 'tmin', 'tmax', 'velmedia', 'hrMedia', 'hrMax', 'hrMin', 'prec']]
df = df.rename(columns={
    "tmed": "temperature",
    "tmin": "temperature_min",
    "tmax": "temperature_max",
    "velmedia": "wind_speed",
    "hrMedia": "humidity",
    "hrMin": "humidity_min",
    "hrMax": "humidity_max",
    "prec": "precipitation",
})

# Reemplazar comas por puntos y convertir a numérico
df = df.map(lambda x: float(x.replace(',', '.')) if isinstance(x, str) else x)

df_aemet = df

In [ ]:
df_om.head()

In [ ]:
df_aemet.head()

In [ ]:
# Asegurar que ambas series tienen las mismas fechas para comparar
df_combined = pd.merge(df_aemet, df_om, on="Fecha", suffixes=("_AEMET", "_OM"))
print("\nDatos combinados:")
df_combined.head()

---

#### **Paso 3: Visualizar las series temporales por fuente**

In [ ]:
# Comparación de Temperatura
plt.figure(figsize=(12, 6))
plt.plot(df_combined.index, df_combined["temperature_AEMET"], label="AEMET")
plt.plot(df_combined.index, df_combined["temperature_OM"], label="Open-Meteo")
plt.title("Comparación de Temperatura")
plt.xlabel("Fecha")
plt.ylabel("Temperatura (°C)")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Comparación de todas las variables (Temperatura, Humedad, Viento, Precipitación)
variables = ["temperature", "humidity", "wind_speed", "precipitation"]
for variable in variables:
    plt.figure(figsize=(12, 6))
    plt.plot(df_combined.index, df_combined[f"{variable}_AEMET"], label=f"{variable} - AEMET")
    plt.plot(df_combined.index, df_combined[f"{variable}_OM"], label=f"{variable} - Open-Meteo")
    plt.title(f"Comparación de {variable}")
    plt.xlabel("Fecha")
    plt.ylabel(variable)
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

---

#### **Paso 4: Histograma Comparativo de Temperatura**

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df_combined["temperature_AEMET"], label="AEMET", color="blue", kde=True)
sns.histplot(df_combined["temperature_OM"], label="Open-Meteo", color="orange", kde=True)
plt.title("Distribución de Temperatura - AEMET vs Open-Meteo")
plt.xlabel("Temperatura (°C)")
plt.ylabel("Frecuencia")
plt.legend()
plt.tight_layout()
plt.show()

---

### **Paso 5: Análisis de Estadísticas Descriptivas**

In [ ]:
print("\nEstadísticas descriptivas de AEMET:")
df_combined.filter(like="_AEMET").describe()

In [ ]:
print("\nEstadísticas descriptivas de Open-Meteo:")
df_combined.filter(like="_OM").describe()